In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import cv2
import skimage.exposure as sk


if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate



mat = loadmat('MR_data.mat')
I_noisefree = mat['I_noisefree']
I_noisy1=mat['I_noisy1']
I_noisy2=mat['I_noisy2']
I_noisy3=mat['I_noisy3']
I_noisy4=mat['I_noisy4']


images = [I_noisefree,I_noisy1,I_noisy2,I_noisy3,I_noisy4]

def showImages(org,conv):
  fig,axs = plt.subplots(1,3)
  axs[0].imshow(org,'gray')
  axs[0].set_title('original')
  axs[1].imshow(conv,'gray')
  axs[1].set_title('conv')
  diff = abs(org-conv)
  axs[2].imshow(diff,'gray')
  axs[2].set_title('diff')
  plt.show()
  



# laplacian = np.array((
# 	[0, 1, 0],
# 	[1, -4, 1],
# 	[0, 1, 0]), dtype="int")
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
  


def convolve(img,kernel):
  imgH,imgW = img.shape
  kH,kW = kernel.shape

  pad = (kW-1)//2

  img = cv2.copyMakeBorder(img,pad,pad,pad,pad,cv2.BORDER_REPLICATE)
  out = np.zeros((imgH,imgW),dtype='float32')

  for y in np.arange(pad,imgH+pad):
    for x in np.arange(pad,imgW+pad):
      inner = img[y-pad:y+pad+1, x-pad:x+pad+1]
      k=(inner*kernel).sum()

      out[y-pad,x-pad]=k

  # out = cv2.rescale_intensity(out,in_range=(0,255))
  out=sk.rescale_intensity(out, in_range=(0, 255))
  out = (out*255).astype('uint8')

  return out



# out1 = convolve(images[0],laplacian)
# showImages(images[0],out1)
kernel = fgaussian(5,0.5)

for i in range(len(images)):
  out = convolve(images[i],kernel)
  showImages(images[i],out)


def evalPx(window,kernel,sig):
  kH,kW = kernel.shape
  px=0
  for i in range(kH):
    for j in range(kW):
      y = window[i][j]-kernel[i][j]
      gamma = np.exp(-(y**2)/(2*sig**2))
      px += window[i][j]*kernel[i][j]*gamma
  return px

def bilateral(img,kernel,sig):
  imgH,imgW = img.shape
  kH,kW = kernel.shape

  pad = (kW-1)//2

  img = cv2.copyMakeBorder(img,pad,pad,pad,pad,cv2.BORDER_REPLICATE)
  out = np.zeros((imgH,imgW),dtype='float32')

  for y in np.arange(pad,imgH+pad):
    for x in np.arange(pad,imgW+pad):
      inner = img[y-pad:y+pad+1, x-pad:x+pad+1]
    

      out[y-pad,x-pad]=evalPx(inner,kernel,sig)

  out=sk.rescale_intensity(out, in_range=(0, 255))
  out = (out*255).astype('uint8')

  return out

for i in range(len(images)):
  out = bilateral(images[i],kernel,900)
  showImages(images[i],out)